# Calibrating the CATS model against the US economy using black-it

<table style="font-size: 110%">
    <tr style="background-color: white;">
    <td width="50%" style="vertical-align: top; text-align: left">
        We will use black-it to calibrate CATS (Complex Adaptive Trivial System) a well-known macroeconomic model that our team reimplemented in Julia.<br><br>
        The high level structure of the model is illustrated on the right: <span style="background-color: #D3E7C6; font-weight: bold;">Agent classes</span> are represented in green ovals, interaction types are specified in rectangles: <span style="background-color: #FFE699; font-weight: bold;">markets</span> are yellow, <span style="background-color: #F8CBAD; font-weight: bold;">bank deposits</span> are peach.<br><br>
        The directions of the arrows indicate the flow of the specific good e.g., consumption goods are acquired by households from C-firms, while labour is acquired by firms from households.<br><br>
        The model has <strong>28 input parameters</strong> and <strong>16 output parameters</strong>.</td>
    <td style="vertical-align: top; text-align: left"><div style="text-align: center">CATS high level architecture:</div><img src="data/agenc-architecture.png" alt="AgenC architechture" style="width: 500px;"/></td>
    </tr>
</table>

In [ ]:
import abm_models
import cats_util
import numpy as np

from black_it.calibrator import Calibrator
from black_it.loss_functions.msm import MethodOfMomentsLoss
from black_it.samplers.best_batch import BestBatchSampler
from black_it.samplers.random_forest import RandomForestSampler
from black_it.samplers.halton import HaltonSampler
from black_it.utils.time_series import log_and_hp_filter, diff_log_demean_filter

In [ ]:
# import the target data, the US economy from 1947 to 2019
real_data = np.genfromtxt('FRED_data.txt')
cats_util.plot_cats_output(real_data)

In [ ]:
cats_model = abm_models.make(
    "CatsModel", 
    path_to_cats_directory="/home/black-it-experiments/disk/black-it-demo/AgenC",
    W=500,
    F=50,
    N=10, 
    warm_up_samples=200,
    output_variables=[
      "Y_real",
      "gdp_deflator",
      "Investment",
      "consumption",
      "Un",
    ]
)

In [ ]:
cats_util.cleanup_output_dir()

In [ ]:
# try to run the ABM with some parameters
output = cats_model.run(
    params={
        "Iprob": 0.25,
        "chi": 0.05,
        "delta": 0.5,
        "inventory_depreciation": 0.3,
        "mu": 1.2,
        "p_adj": 0.1,
        "phi": 0.02,
        "q_adj": 0.9,
        "tax_rate": 0.05,
        "theta": 0.05,
        "xi": 0.96,
    }, 
    nb_samples=281,
    seed=0,
)

In [ ]:
# plot one of the obtained series
cats_util.plot_gdp(output)

In [ ]:
# define a simple model wrapper
def model_wrapper(theta, N, seed):
    return cats_model.run(
        params=cats_util.found_params | {
            "Iprob": theta[0],
            "inventory_depreciation": theta[1],
            "mu": theta[2],
        },
        nb_samples=N, 
        seed=seed,
    )

In [ ]:
# define a method of moments loss function with some coordinate filters
coordinate_filters = [
    log_and_hp_filter, 
    diff_log_demean_filter, 
    log_and_hp_filter, 
    log_and_hp_filter, 
    None,
]

loss = MethodOfMomentsLoss(
    coordinate_filters=coordinate_filters,
    covariance_mat="identity",
    standardise_moments=True,
)

In [ ]:
# define the simulation bounds and precisions
bounds = np.array([
        [0.0, 0.5],
        [0.0, 1.0],
        [1.0, 1.5]]).T

precisions = [0.001] * 3

In [ ]:
# number of model realizations to run for each set of parameters
ensemble_size = 4

# initialize the Calibrator
cal = Calibrator(
    samplers=[HaltonSampler(batch_size=8)], # let's start with a simple sampler to bootstrap the search
    real_data=real_data,
    sim_length=800,
    model=model_wrapper,
    parameters_bounds=bounds,
    parameters_precision=precisions,
    ensemble_size=ensemble_size, 
    loss_function=loss,
    saving_folder=cats_util.saving_folder,
    random_state=0,
    n_jobs=32,
)

In [ ]:
# sample some initial seed points uniformly using a Halton sampler
params, losses = cal.calibrate(1)

In [ ]:
# change the sampler to a more adaptive one
cal.set_samplers([
    RandomForestSampler(batch_size=8),
    BestBatchSampler(batch_size=8),
])

In [ ]:
# calibrate for a number of epochs
cal.calibrate(5)

In [ ]:
# find the best loss index and print the best loss
best_idx = np.argmin(cal.losses_samp)
print(f"Best loss index: {best_idx} (out of {len(cal.losses_samp)} simulations)")
print(f"Best loss value: {cal.losses_samp[best_idx]:.2f}")

In [ ]:
# retrieve the best loss simulations
best_sim = cal.series_samp[best_idx]

In [ ]:
# compare real and simulated moments
cats_util.compare_moments(real_data, best_sim, ensemble_size, coordinate_filters)

- 0-3: mean, variance, skewness and kurtosis
- 4-8: autocorrelations of increasing time lags
- 9-12: mean, variance, skewness and kurtosis of the differentiated time series
- 13-17: autocorrelations of the differentiated time series.